# Plenoptic-Toolbox To D-NeRF dataset format


#### Notes on D-NeRF dataset:

In the nerfstudio DNeRF parser, `camera_angle_x` parameter (found in the transforms files) has a $1/2*tanh(0.5*x)$ relationship with the focal length. As the PTB dataset does not come with this, I use the nerfstudio repository to perform camera calibration using the 0th frame of all the videos. This will give us `(f_x, f_y, c_x, c_y)` (the focal and pixel area intrinsics) which replace the `camera__angle_x` parametr present in the initial D-NeRF dataset

Otherwise the `rotation` parameter found for each frame doesn't seem to have any impact on performance (at least in the tests I ran with K-Planes.

---
# Configuration
---

1. Set-Up directories
2. Filter through the PTB dataset and re-construct the transformation matrix.
3. Store training data
4. Repeat for Test and Val data

In [1]:
import os

# Set the plenoptic toolbox (ptb) config and video files
config_fp ='data/plenoptic_toolbox/161029_sports1/calibration_161029_sports1.json'
video_fp = 'data/plenoptic_toolbox/161029_sports1/hdVideos/'
# Set the video type
if 'hdVideos' in video_fp:
    camType = 'hd'
elif 'vgaVideos' in video_fp:
    camType = 'vga'
else: # default currently assumes VGA camera dataset
    camType = 'vga'
# Set the output folder
output_fp = 'data/plenoptic_toolbox/161029_sports1/dnerf/'

# Assert input data exists
assert os.path.exists(config_fp), AssertionError(f'Config Non-Existent : {config_fp}')
assert os.path.exists(video_fp), AssertionError(f'Config Non-Existent : {video_fp}')


# Construct folders (replace existing folders)
os.makedirs(output_fp, exist_ok=True) # create root folder
train_im_fp = output_fp+'train/'
test_im_fp = output_fp+'test/'
val_im_fp = output_fp+'val/'
os.makedirs(train_im_fp, exist_ok=True) # create train, test and val folders
os.makedirs(test_im_fp, exist_ok=True)
os.makedirs(val_im_fp, exist_ok=True)


In [2]:
import json
import numpy as np

with open(config_fp, 'r') as fp:
    config = json.load(fp)

# Initialise used_frame dict
used_frames = {} # dict for storing "[camera]":[frame0, frame 12, frame 129, ...] (frames used in train and/or test)

# Filter through camera data and store video pose data
cameras = []
for cam in config['cameras']:
    if cam['type'] == camType:
        # Construct a 4x4 transformation matrix
        R = np.matrix(cam['R'])
        t = np.array(cam['t'])
        transformation_matrix = np.eye(4)
        transformation_matrix[:3, :3] = R.transpose()
        cc = (-R.transpose()*t)
        transformation_matrix[:3, 3] =cc.flatten()/80. # Here we re-scale scene from 300x300x300 to 4x4x4 (not exactly 4 but nearly)
        
        # Store necessary data
        cameras.append({
            "name":cam['name'],
            "transformation_matrix":transformation_matrix
        })
        # Initialise store for each camera
        used_frames[cam['name']] = []

# Initialise the Dictionaries that will hold our training, testing and validation data
training_data = {
    "camera_angle_x":0.1,
    "w": 1920, # Intrinsics generate by nerfstudio
    "h": 1080,
    "fl_x": 1468.757320423469,
    "fl_y": 1471.8049431235877,
    "cx": 938.9616276596577,
    "cy": 562.453456691303,
    "k1": -0.2530330313118452,
    "k2": 0.1464548200867903,
    "p1": 0.00011976648210367369,
    "p2": 6.6743379744858146e-06,
    "camera_model": "OPENCV",
    "frames":[]
}
testing_data = {
    "camera_angle_x":0.1,
    "w": 1920, # Intrinsics generate by nerfstudio
    "h": 1080,
    "fl_x": 1468.757320423469,
    "fl_y": 1471.8049431235877,
    "cx": 938.9616276596577,
    "cy": 562.453456691303,
    "k1": -0.2530330313118452,
    "k2": 0.1464548200867903,
    "p1": 0.00011976648210367369,
    "p2": 6.6743379744858146e-06,
    "camera_model": "OPENCV",
    "frames":[]
}
validation_data = {
    "camera_angle_x":0.1,
    "w": 1920, # Intrinsics generate by nerfstudio
    "h": 1080,
    "fl_x": 1468.757320423469,
    "fl_y": 1471.8049431235877,
    "cx": 938.9616276596577,
    "cy": 562.453456691303,
    "k1": -0.2530330313118452,
    "k2": 0.1464548200867903,
    "p1": 0.00011976648210367369,
    "p2": 6.6743379744858146e-06,
    "camera_model": "OPENCV",
    "frames":[]
}

# Train-Test-Val Split

1. We select one camera for validation, one camera for testing and the remainder for training.
2. Additionally 10 frames from each training camera will be selected for validation and testion (5 each)

In [3]:
# Extract training data
import cv2
import random

frameNum = 0
camNum = len(cameras)

# Video is 180 frames
total_frames = 20

# Arbitrary selection
testCamera = 2
valCamera = 18
assert (testCamera < camNum) and (valCamera < camNum), AssertionError('Either testCamera or valCamera is set incorrectly (above camNum)')
assert (testCamera >= 0) and (valCamera >= 0), AssertionError('Either testCamera or valCamera is set incorrectly (below 0)')

rotation = 0.3141592653589793

for i, cam in enumerate(cameras):
    # Construct filepath to video
    name_start = camType+'_'
    fp = video_fp+name_start+cam['name']+'.mp4'
    assert os.path.exists(fp), AssertionError(f'Video Not Found: {fp}')
    
    # Get Number of cameras and number of frames
    video = cv2.VideoCapture(str(fp))

    # Get the total number of frames for the video
    # total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT)) # Uncomment if you want to use the whole video

    # Set the destination folder for images
    if i == testCamera:
        folder = 'test/'
    elif i == valCamera:
        folder = 'val/'
    else:
        folder = 'train/'

        # Determine the frames from each video to associate with validation or testing
        test_val_frames = random.sample(range(total_frames), 10)
        test_frames = test_val_frames[:5]
        val_frames = test_val_frames[5:]
    
    # Cycle through each frame as store w.r.t val_frames, test_frames and the selected camera
    for frameNum in range(total_frames):
        video.set(cv2.CAP_PROP_POS_FRAMES, frameNum)
        ret, frame = video.read()

        # Set the destination folder of the images
        localDestinationFolder = folder
        if folder == 'train/':
            if frameNum in test_frames:
                localDestinationFolder = 'test/'
            elif frameNum in val_frames:
                localDestinationFolder = 'val/'
        destinationFolder = output_fp + localDestinationFolder # E.g. 'dnerf/[val/train/test]/'
        cv2.imwrite(destinationFolder+f'{cam["name"]}_{frameNum}.png', frame)
        
        frameData = {
            "file_path":f'{localDestinationFolder}{cam["name"]}_{frameNum}',
            "rotation": rotation,
            "time": float(frameNum/total_frames),
            "transform_matrix":cam["transformation_matrix"].tolist()
        }
        
        if i == testCamera:
            testing_data["frames"].append(
                frameData
            )
        elif i == valCamera:
            validation_data["frames"].append(
                frameData
            )
        else:
            training_data["frames"].append(
                frameData
            )

In [4]:
with open(output_fp+'transforms_train.json','w') as fp:
    json.dump(training_data, fp)
with open(output_fp+'transforms_test.json','w') as fp:
    json.dump(testing_data, fp)
with open(output_fp+'transforms_val.json','w') as fp:
    json.dump(validation_data, fp)